In [135]:
from project.utils import get_pandas
import pandas as pd
import os

# Merging SDES database with Profeel to get households information
The method is to match the sdes archetypes to a Profeel one according to the values for the three index levels: Housing types, Energy performance, heating energy.
And then we calculate among a group the weight of the sdes archetypes in terms of their sdes's size estimate. Finally, this weights allow us to get the size of each lines in terms of Profeel stock.

The merging rule is to conserve, from the database of origin (SDES), the proportions of each household characteristics among a group of buildings which have same basic building characteristics.

To be change:
- We can choose another profeel database, with no ventilation or with other energy performance estimates for example
- We can put the output in another place than analysis/preprocessing/building stock/output

In [136]:
profeel = get_pandas('project/input/stock/building_stock_profeel_vent.csv', lambda x: pd.read_csv(x, index_col=[1]).squeeze())
sdes = get_pandas('project/input/stock/building_stock_sdes2018_aggregated.csv', lambda x: pd.read_csv(x, index_col=[0, 1, 2]).squeeze())

profeel = profeel.rename(columns={'calculated_epc':'Energy performance'})
profeel['Heating energy'] = profeel['Heating system'].str.split('-').str[0]
profeel = profeel.set_index(['Heating energy', 'Energy performance'], append=True)

In [137]:
merged = profeel.merge(sdes, how='inner', on=['Housing type', 'Energy performance', 'Heating energy'])
# Adding a row specific index level
r_position = pd.Series(range(0,merged.shape[0]), name='r_position')
merged = merged.set_index(r_position, append=True)

# Weight of each lines in terms of sdes's sizes (y) among a profeel archetype uniquely define by its class label
merged = merged.set_index('Class', append=True)
merged['Stock buildings_y'] = merged['Stock buildings_y'].T / merged.groupby(level='Class')['Stock buildings_y'].sum()

# Multiplying the weights to the profeel's stocks (x)
merged['Stock buildings'] = (merged['Stock buildings_y'].T * merged['Stock buildings_x']).T
cleaned = merged.drop(labels=['Stock buildings_x', 'Stock buildings_y'], axis=1)
cleaned

Heating system  \
Housing type  Energy performance Heating energy r_position Class                                   
Single-family F                  Wood fuel      0          MI-1        Wood fuel-Standard boiler   
                                                1          MI-1        Wood fuel-Standard boiler   
                                                2          MI-1        Wood fuel-Standard boiler   
                                                3          MI-1        Wood fuel-Standard boiler   
                                                4          MI-1        Wood fuel-Standard boiler   
...                                                                                          ...   
Multi-family  B                  Electricity    4675       LC24   Electricity-Performance boiler   
                                                4676       LC24   Electricity-Performance boiler   
                                                4677       LC24   Electricity-Performance boiler   
                                                4678       LC24   Electricity-Performance boiler   
                                                4679       LC24   Electricity-Performance boiler   

                                                                  Wall  Floor  \
Housing type  Energy performance Heating energy r_position Class                
Single-family F                  Wood fuel      0          MI-1   2.50   2.00   
                                                1          MI-1   2.50   2.00   
                                                2          MI-1   2.50   2.00   
                                                3          MI-1   2.50   2.00   
                                                4          MI-1   2.50   2.00   
...                                                                ...    ...   
Multi-family  B                  Electricity    4675       LC24   0.34   0.25   
                                                4676       LC24   0.34   0.25   
                                                4677       LC24   0.34   0.25   
                                                4678       LC24   0.34   0.25   
                                                4679       LC24   0.34   0.25   

                                                                  Roof  \
Housing type  Energy performance Heating energy r_position Class         
Single-family F                  Wood fuel      0          MI-1    2.5   
                                                1          MI-1    2.5   
                                                2          MI-1    2.5   
                                                3          MI-1    2.5   
                                                4          MI-1    2.5   
...                                                                ...   
Multi-family  B                  Electricity    4675       LC24    0.2   
                                                4676       LC24    0.2   
                                                4677       LC24    0.2   
                                                4678       LC24    0.2   
                                                4679       LC24    0.2   

                                                                  Windows  \
Housing type  Energy performance Heating energy r_position Class            
Single-family F                  Wood fuel      0          MI-1       2.4   
                                                1          MI-1       2.4   
                                                2          MI-1       2.4   
                                                3          MI-1       2.4   
                                                4          MI-1       2.4   
...                                                                   ...   
Multi-family  B                  Electricity    4675       LC24       1.6   
                                                4676       LC24       1

In [138]:
# Checks
print("profeel building stock: {}".format(profeel['Stock buildings'].sum()))
print("sdes building stock: {}".format(sdes['Stock buildings'].sum()))
print("The number of new lines for each profeel archetypes: {}".format(merged.shape[0]/profeel.shape[0]))
print("merged building stock: {}".format(merged['Stock buildings_x'].sum()/(merged.shape[0]/profeel.shape[0])))
print("output building stock: {}".format(cleaned['Stock buildings'].sum()))
print("building stock loss: {}".format(profeel['Stock buildings'].sum() - cleaned['Stock buildings'].sum()))

profeel building stock: 35172600
sdes building stock: 26698690.691502385
The number of new lines for each profeel archetypes: 120.0
merged building stock: 35172600.0
output building stock: 35172600.0
building stock loss: 0.0


In [139]:
path_output = os.path.join(os.getcwd(), 'preprocessing/building_stock/output')
cleaned.to_csv(os.path.join(path_output, 'building_stock_profeel_hhdata.csv'))